In [22]:
import rasterio as rio
import matplotlib.pyplot as plt 
import matplotlib.patches as patches

from matplotlib.colors import Normalize
import numpy as np
import numpy.matlib
from scipy import interpolate
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as st
import scipy
import os, sys, pickle, gzip
import datetime
import geopy.distance
import xarray as xr
import pandas as pd
import rasterio
import geopandas as gpd
import shapely.geometry
import shapely.ops
import xesmf as xe
import cartopy
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.util import add_cyclic_point
import itertools
import random
import metpy
from metpy.plots import USCOUNTIES

from dask.diagnostics import ProgressBar
from dask.distributed import Client, progress

import warnings
warnings.filterwarnings('ignore')

In [23]:
run ../util/setupConsole_su

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
cmip6_models = ['bcc-csm2-mr', 'bcc-esm1', 'canesm5', \
                'kace-1-0-g', 'ipsl-cm6a-lr', 'miroc6', \
                'mri-esm2-0', 'noresm2-lm']

cmip6_lat = np.arange(-90, 90, 1.5)
cmip6_lon = np.arange(0, 360, 1.5)


In [25]:
dirAgData = '/home/edcoffel/drive/MAX-Filer/Research/Climate-01/Personal-F20/edcoffel-F20/data/projects/ag-land-climate'
dirEra5 = '/home/edcoffel/drive/MAX-Filer/Research/Climate-02/Data-02-edcoffel-F20/ERA5'
dirEra5Land = '/home/edcoffel/drive/MAX-Filer/Research/Climate-02/Data-02-edcoffel-F20/ERA5-Land'
dirCMIP6 = '/home/edcoffel/drive/MAX-Filer/Research/Climate-02/Data-02-edcoffel-F20/CMIP6'
dirHeatData = '/home/edcoffel/drive/MAX-Filer/Research/Climate-01/Personal-F20/edcoffel-F20/data/projects/2021-heat'
dirAg6 = '/home/edcoffel/drive/MAX-Filer/Research/Climate-01/Personal-F20/edcoffel-F20/research/2020-ag-cmip6'

In [26]:
era5_max_deciles = xr.open_dataset('%s/era5_tw_max_deciles.nc'%dirHeatData)
lat = era5_max_deciles.latitude.values
lon = era5_max_deciles.longitude.values

In [27]:
# era5_huss_on_txx = xr.open_mfdataset('output/huss_on_tx/huss_on_txx_*.nc', concat_dim='time', combine='nested')
# era5_huss_on_txx = era5_huss_on_txx.rename({'__xarray_dataarray_variable__':'huss_on_txx'})
# era5_huss_on_txx = era5_huss_on_txx.where(land_sea_mask_binary)
# era5_huss_on_txx = era5_huss_on_txx.sel(latitude=slice(60,-60))
# era5_huss_on_txx.mean(dim='time').to_netcdf('era5_huss_on_txx_mean.nc')

In [28]:
# tx_era5 = xr.open_mfdataset('%s/daily/tasmax_*.nc'%dirEra5)
# tx_era5 = tx_era5.sel(time=slice('1981', '2020'))
# tx_era5_mean = tx_era5.mean(dim='time')
# sm_era5_mean.to_netcdf('era5_sm_mean.nc')

In [29]:
land_sea_mask = xr.open_dataset('%s/land-sea-mask.nc'%dirEra5)
land_sea_mask.load()
land_sea_mask = land_sea_mask.lsm.mean(dim='time')
land_sea_mask_binary = land_sea_mask > 0.1


In [30]:
# from datetime import datetime
# def add_time_dim(xda):
#     xda = xda.expand_dims(time = [datetime.now()])
#     return xda

# time_dim = pd.date_range("1981-01-01", "2020-12-31", freq="AS")
# era5_var_deciles = xr.open_mfdataset('deciles/tw/era5_tw_max_deciles_warm_season_*_new.nc', preprocess = add_time_dim, concat_dim='time')
# era5_var_deciles['time'] = time_dim
# era5_var_deciles = era5_var_deciles.chunk(chunks={"time": -1, "latitude": 50, "longitude": 50})
# era5_var_deciles = era5_var_deciles.chunk({"time": -1})
# era5_var_deciles = era5_var_deciles.chunk({"quantile": -1})


from datetime import datetime
def add_time_dim(xda):
    xda = xda.expand_dims(time = [datetime.now()])
    return xda

time_dim = pd.date_range("1981-01-01", "2021-12-31", freq="AS")
# era5_var_deciles = xr.open_mfdataset('output/huss_on_tx/huss_on_warm_season_tx_deciles_*_new.nc', preprocess = add_time_dim, combine='nested', concat_dim='time')
era5_var_deciles = xr.open_mfdataset('output/vpd_daily_mean_on_tx/vpd_daily_mean_on_txx*.nc', combine='nested', concat_dim='time')
# era5_var_deciles = era5_var_deciles.rename({'bin':'quantile'})
era5_var_deciles['time'] = time_dim
era5_var_deciles = era5_var_deciles.chunk(chunks={"time": -1, "latitude": 50, "longitude": 50})
era5_var_deciles = era5_var_deciles.chunk({"time": -1})
# era5_var_deciles = era5_var_deciles.chunk({"quantile": -1})

# era5_var_deciles2 = xr.open_mfdataset('output/tw_max_on_tw/tw_max_on_txx*.nc', combine='nested', concat_dim='time')
# # era5_var_deciles = era5_var_deciles.rename({'bin':'quantile'})
# era5_var_deciles2['time'] = time_dim
# era5_var_deciles2 = era5_var_deciles2.chunk(chunks={"time": -1, "latitude": 50, "longitude": 50})
# era5_var_deciles2 = era5_var_deciles2.chunk({"time": -1})


In [31]:
era5_var_deciles

,Array,Chunk
Bytes,340.54 MB,820.00 kB
Shape,"(41, 721, 1440)","(41, 50, 50)"
Count,1866 Tasks,435 Chunks
Type,float64,numpy.ndarray


In [32]:
# era5_var_deciles = era5_var_deciles.rename({'e':'evap_on_txx'})
era5_var_deciles = era5_var_deciles.rename({'__xarray_dataarray_variable__':'vpd_daily_mean_on_txx'})
# era5_var_deciles2 = era5_var_deciles2.rename({'__xarray_dataarray_variable__':'tw_on_txx'})


In [33]:
# era5_var_deciles = era5_var_deciles.tw_on_txx - era5_var_deciles2.tw_on_txx

In [34]:
land_sea_mask_binary = land_sea_mask_binary.broadcast_like(era5_var_deciles)
era5_var_deciles_masked = era5_var_deciles.where(land_sea_mask_binary)

In [35]:
# era5_var_deciles_masked = era5_var_deciles_masked.sel(quantile=np.arange(0, 101, 5)/100, method='nearest')

In [36]:
era5_var_deciles_masked


,Array,Chunk
Bytes,340.54 MB,820.00 kB
Shape,"(41, 721, 1440)","(41, 50, 50)"
Count,3172 Tasks,435 Chunks
Type,float64,numpy.ndarray


In [37]:
def linregress_ufunc(x, y):
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return slope, p_value



def apply_trend_and_pvalue(ds, time_dim='time', x_dim='longitude', y_dim='latitude', bin_dim='quantile'):
    # Extract the year information from the time coordinates
    time_years = ds[time_dim].dt.year.values
    unique_years = np.unique(time_years)
    
    # Broadcast unique_years along the time dimension
    unique_years_broadcasted = xr.DataArray(unique_years, dims=[time_dim], coords={time_dim: ds[time_dim]})
    
    trend, p_value = xr.apply_ufunc(
        linregress_ufunc,
        unique_years_broadcasted,
        ds,
        input_core_dims=[[time_dim], [time_dim]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float]
    )

    trend = trend.rename("vpd_daily_mean_on_txx_trend")
    p_value = p_value.rename("vpd_daily_mean_on_txx_p_value")
    
    return xr.merge([trend, p_value])





In [38]:
# temperature_trend = apply_trend_and_pvalue(era5_var_deciles_masked["mx2t"])
# evap_on_txx_trend = apply_trend_and_pvalue(era5_var_deciles_masked['tx_on_txx'])
evap_on_txx_trend = apply_trend_and_pvalue(era5_var_deciles_masked['vpd_daily_mean_on_txx'])

In [39]:
# evap_on_warm_season_tw_trend.to_netcdf('era5_evaporation_on_warm_season_tw_trends_by_decile_bootstrap.nc')

In [40]:
# trend_ds = xr.Dataset({"tw_trend_deciles": temperature_trend})

In [41]:
client = Client()

In [42]:
# with ProgressBar():
# temperature_trend.compute().to_netcdf("era5_tw_warm_season_trends_by_decile_new.nc")
evap_on_txx_trend.compute().to_netcdf("era5_vpd_daily_mean_on_txx_trends.nc")
#     dask.array.store(trend_ds["tw_trend_deciles"].data, output_ds["tw_trend_deciles"].data, lock=False)